<a href="https://colab.research.google.com/github/arnabghorui/smai_project/blob/main/SMAI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TF IDF & MLP

Here we are given two articles and we have to check similarity between them,
There are many metrics of similarity, <br>
<br>
One of them is TF IDF ( Total Frequency * Inverse Document Frequency). In this metric we multiply the number of times a word is present in a document to the inverse of the number of the articles that the word appears in. The inverse part shows the overall importance of the word and the frequency part shows how important the given word is in the context of the present article. <br>
<br>


###Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/Project/train_data.json')
train_data = json.load(f)
g= open('/content/drive/MyDrive/Colab Notebooks/Project/test_data.json')
test_data = json.load(g)

In [4]:
print(test_data[0].keys())

print(len(test_data))

dict_keys(['pair_id', 'title_1', 'title_2', 'text_1', 'text_2', 'tags_1', 'tags_2', 'meta_lang_1', 'meta_lang_2', 'score'])
4316


After Filtering out the english articles - we compare the two articles in a given element. 

For that we append the two articles with keys : text_1 and text_2 

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from string import punctuation


english_articles_train = []
english_articles_test = []


for d in train_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_train.append(d)

for d in test_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_test.append(d)



total_train_articles = []
total_test_articles = []

y_train = []
y_test = []

for eng in english_articles_train:
  total_train_articles.append(eng['text_1'])
  total_train_articles.append(eng['text_2'])
  y_train.append(eng['score'])

for eng in english_articles_test:
  total_test_articles.append(eng['text_1'])
  total_test_articles.append(eng['text_2'])
  y_test.append(eng['score'])


'''
art1 = art1.split()

art2 = art2.split()

for i in range(len(art1)):
    art1[i] = ''.join([letter for letter in art1[i] if letter not in punctuation])

for i in range(len(art2)):
    art2[i] = ''.join([letter for letter in art2[i] if letter not in punctuation])
'''

#########################################################



tf = TfidfVectorizer(min_df =2)

x_train = tf.fit(total_train_articles)
x_train = tf.transform(total_train_articles)
x_test = tf.transform(total_test_articles)


print(x_train.shape)

print(type(x_train))



X_text_1_train = []
X_text_2_train = []
X_text_1_test = []
X_text_2_test = []


print(x_test.shape)




for j in range(1287):
  X_text_1_train.append(x_train[2*j])
  X_text_2_train.append(x_train[2*j+1])

for j in range(236):
  X_text_1_test.append(x_test[2*j])
  X_text_2_test.append(x_test[2*j+1])



(2574, 25972)
<class 'scipy.sparse.csr.csr_matrix'>
(472, 25972)


In [ ]:
from scipy.sparse import csr_matrix
 

X1_train = []
X2_train = []
X1_test = []
X2_test = []

for i in range(1287):
  X1_train.append(np.array(csr_matrix.todense(X_text_1_train[i])))
  X2_train.append(np.array(csr_matrix.todense(X_text_2_train[i])))

for i in range(236):
  X1_test.append(np.array(csr_matrix.todense(X_text_1_test[i])))
  X2_test.append(np.array(csr_matrix.todense(X_text_2_test[i])))


y_train = np.array(y_train)
y_test = np.array(y_test)

#scor1 = np.reshape(scor1,(1287,1))
X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)

#rows1 = np.reshape()

X1_train = X1_train.reshape(1287,25972)
X2_train = X2_train.reshape(1287,25972)
X1_test = X1_test.reshape(236,25972)
X2_test = X2_test.reshape(236,25972)



#Model

This is the Siamese implementation of the 

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, LSTM, Input, concatenate

input_1 = Input(shape=(25972, ))
dense_1 = Dense(1000, )(input_1)

input_2 = Input(shape=(25972, ))
dense_2 = Dense(1000, )(input_2)

merge = concatenate([dense_1, dense_2])

dense_3  = Dense(200,)(merge)
dense_4  = Dense(50,)(dense_3)
dense_all = Dense(1,)(dense_4)
#second_dense = Dense(2, activation='softmax')(merge)

model1 = Model(inputs=[input_1, input_2], outputs=dense_all)
optimizer = keras.optimizers.Adam()  #learning_rate=0.001
model1.compile(loss='mse', optimizer = optimizer) #, metrics=['RootMeanSquaredError']
print(model1.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 dense (Dense)                  (None, 1000)         25973000    ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 1000)         25973000    ['input_2[0][0]']                
                                                                                              

#Model 2
Model Changed from concatenate(dense1, dense2) to (desnse1+dense2, dense1-dense2)

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, LSTM, Input, concatenate

input_1 = Input(shape=(25972, ))
dense_1 = Dense(1000, )(input_1)

input_2 = Input(shape=(25972, ))
dense_2 = Dense(1000, )(input_2)

merge = concatenate([dense_1+dense_2, dense_1-dense_2])

dense_3  = Dense(200,)(merge)
dense_4  = Dense(50,)(dense_3)
dense_all = Dense(1,)(dense_4)
#second_dense = Dense(2, activation='softmax')(merge)

model2 = Model(inputs=[input_1, input_2], outputs=dense_all)
optimizer = keras.optimizers.Adam()  #learning_rate=0.001
model2.compile(loss='mse', optimizer = optimizer) #, metrics=['RootMeanSquaredError']
print(model2.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 1000)         25973000    ['input_3[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 1000)         25973000    ['input_4[0][0]']                
                                                                                            

In [ ]:
history1 = model1.fit([X1_train, X2_train], y_train, epochs=10)
history2 = model2.fit([X1_train, X2_train], y_train, epochs=10,validation_split=0.1)

Epoch 1/10
41/41 [==============================] - 1s 19ms/step - loss: 0.0606
Epoch 2/10
41/41 [==============================] - 1s 17ms/step - loss: 0.0542
Epoch 3/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0545
Epoch 4/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0580
Epoch 5/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0661
Epoch 6/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0659
Epoch 7/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0748
Epoch 8/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0487
Epoch 9/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0468
Epoch 10/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0383
Epoch 1/10
37/37 [==============================] - 1s 20ms/step - loss: 0.0570 - val_loss: 0.0436
Epoch 2/10
37/37 [==============================] - 1s 15ms/step - loss: 0.0729 - val_loss: 0.0385
E

In [ ]:
y_pred_1 = model1.predict([X1_test,X2_test])
y_pred_2 = model2.predict([X1_test,X2_test])

8/8 [==============================] - 0s 4ms/step


In [ ]:
print(y_pred_1)
print(y_pred_2)

[[2.9809422]
 [3.2298825]
 [2.4723365]
 [2.8093872]
 [2.3284345]
 [3.0277693]
 [3.5084498]
 [1.8500911]
 [2.5189412]
 [3.2646132]
 [2.5958812]
 [2.8218825]
 [3.672472 ]
 [3.3527753]
 [3.300978 ]
 [2.361161 ]
 [3.9347577]
 [2.1375806]
 [2.4439423]
 [3.9231982]
 [2.3683817]
 [2.8471456]
 [2.8148837]
 [2.6161768]
 [1.8584085]
 [2.4444542]
 [2.5311158]
 [2.7384303]
 [3.3683312]
 [3.6996386]
 [2.9658577]
 [3.0264044]
 [2.6420095]
 [3.6909537]
 [4.1573653]
 [2.7990744]
 [4.1894712]
 [3.7581363]
 [3.6441703]
 [4.47805  ]
 [2.806039 ]
 [2.611947 ]
 [2.4773178]
 [3.1261191]
 [3.4821289]
 [2.3958135]
 [3.2409844]
 [3.4030414]
 [3.4478858]
 [2.4902194]
 [3.4485834]
 [2.975563 ]
 [3.1894436]
 [2.886285 ]
 [2.1637359]
 [2.739224 ]
 [3.3183134]
 [3.5021665]
 [3.5434754]
 [2.016026 ]
 [2.815975 ]
 [2.497089 ]
 [2.9405055]
 [1.8432845]
 [3.3067393]
 [1.8795832]
 [2.39801  ]
 [1.6049811]
 [2.5022113]
 [2.852482 ]
 [3.4018767]
 [2.871321 ]
 [2.617958 ]
 [2.533638 ]
 [3.1353767]
 [3.3081014]
 [3.0931566]

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.feature_selection import r_regression
print('Model 1')
print('The mean absolute error for the model is ', mean_absolute_error(y_test,y_pred_1))
print('The mean squared error for the model is ', mean_squared_error(y_test,y_pred_1))
print('Model 2')
print('The mean absolute error for the model is ', mean_absolute_error(y_test,y_pred_2))
print('The mean squared error for the model is ', mean_squared_error(y_test,y_pred_2))

Model 1
The mean absolute error for the model is  1.1405394834987188
The mean squared error for the model is  1.758964420685308
Model 2
The mean absolute error for the model is  1.1532950859285342
The mean squared error for the model is  1.8045095948697059


In [ ]:
from scipy.stats import pearsonr

print(pearsonr(y_test,y_pred_1))
print(pearsonr(y_test,y_pred_2))

(array([0.24975278465873554], dtype=object), 0.00010530205086652021)
(array([0.24652218067172682], dtype=object), 0.00013010624393758476)


In [ ]:
y_pred_2_train = model2.predict([X1_train,X2_train])
print(pearsonr(y_train,y_pred_2_train))

41/41 [==============================] - 0s 4ms/step
(array([0.9849088105516094], dtype=object), 0.0)
